In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

import os
os.chdir("../")

%load_ext autoreload
%autoreload 2

In [2]:
RANDOM_STATE = 42  # for reproducibility
np.random.seed(RANDOM_STATE)
NUM_ELECTRODES = 64
FS = 240          # Hz
NUM_TRAIN_LETTERS = 85
NUM_TEST_LETTERS = 100
NUM_ROWCOLS = 12    # col:1-6 row:7-12
NUM_REPEAT = 15
SECONDS_TO_SLICE = 0.65    # after the simulation 0.65s data is treated as a sample
DATA_DIR = "./data/raw/BCI_Comp_III_Wads_2004/"

## Prepare data set of Subject A

### mat->subsample->bandpass->normalize along channel

In [3]:
from src import pipelines

A_train = pipelines.signal_mat_sub_band_norm(DATA_DIR+"Subject_A_Train")
A_test = pipelines.signal_mat_sub_band_norm(DATA_DIR + "Subject_B_Test")

In [4]:
X_train = A_train['signal'].reshape([-1,NUM_ELECTRODES,A_train['signal'].shape[-1],1])
X_test = A_test['signal'].reshape([-1,NUM_ELECTRODES,A_test['signal'].shape[-1],1]) 

In [5]:
from keras import layers, models
from keras.utils import to_categorical

Using TensorFlow backend.


In [6]:
y_train = to_categorical(A_train['label'])

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.05,
                                                  random_state=RANDOM_STATE, stratify=y_train)

In [8]:
from src.models.PAMI import CNN_1_P300_PAMI_BCIIII

In [9]:
model_PAMI = CNN_1_P300_PAMI_BCIIII(Ns = 10,seconds_to_slice=SECONDS_TO_SLICE)

In [10]:
model_PAMI.compile(optimizer = 'sgd',loss = 'binary_crossentropy',metrics = ['acc', 'mse'])

In [11]:
from keras.callbacks import EarlyStopping
earlystopping = EarlyStopping(monitor = "val_mean_squared_error",patience = 10)

In [12]:
model_PAMI.fit(x = X_train, y = y_train, batch_size=32, epochs = 1000, callbacks = [earlystopping], 
              validation_data = [X_val,y_val])

Train on 14535 samples, validate on 765 samples
Epoch 1/1000
14535/14535 [==============================] - 77s - loss: 0.4605 - acc: 0.8273 - mean_squared_error: 0.1433 - val_loss: 0.4316 - val_acc: 0.8405 - val_mean_squared_error: 0.1320
Epoch 2/1000
14535/14535 [==============================] - 4s - loss: 0.4489 - acc: 0.8290 - mean_squared_error: 0.1393 - val_loss: 0.4244 - val_acc: 0.8405 - val_mean_squared_error: 0.1298
Epoch 3/1000
14535/14535 [==============================] - 4s - loss: 0.4440 - acc: 0.8290 - mean_squared_error: 0.1378 - val_loss: 0.4170 - val_acc: 0.8405 - val_mean_squared_error: 0.1279
Epoch 4/1000
14535/14535 [==============================] - 4s - loss: 0.4396 - acc: 0.8290 - mean_squared_error: 0.1366 - val_loss: 0.4113 - val_acc: 0.8405 - val_mean_squared_error: 0.1261
Epoch 5/1000
14535/14535 [==============================] - 4s - loss: 0.4350 - acc: 0.8290 - mean_squared_error: 0.1352 - val_loss: 0.4053 - val_acc: 0.8405 - val_mean_squared_error: 0.1

In [13]:
A_test['signal'] = X_test

In [30]:
from src.pipelines import test_pipeline,PARADIGM,accuracy
predictions = test_pipeline(A_test,model_PAMI,14,PARADIGM)

In [31]:
from src.pipelines import accuracy
accuracy(predictions,'WQXPLZCOMRKO97YFZDEZ1DPI9NNVGRQDJCUVRMEUOOOJD2UFYPOO6J7LDGYEGOA5VHNEHBTXOO1TDOILUEE5BFAEEXAW_K4R3MRU')

0.02